### **Ingesting the data to bronze layer using autoloader**

In [0]:
from pyspark.sql.functions import current_timestamp


df_order = (spark.readStream \
            .format("cloudFiles") \
            .option("cloudFiles.format","json") \
            .option("cloudFiles.SchemaLocation","/Volumes/ecommerce/data/streaming_data/Order_checkpoint/") \
            .load("/Volumes/ecommerce/data/streaming_data/Order/")
                )


df_order_items = (spark.readStream \
                    .format("cloudFiles") \
                    .option("cloudFiles.format","json") \
                    .option("cloudFiles.schemaLocation","/Volumes/ecommerce/data/streaming_data/Order_items_checkpoints/") \
                    .load("/Volumes/ecommerce/data/streaming_data/Order_items/")
                        )

df_order = df_order.withColumn("LoadingTime", current_timestamp())
df_order_items = df_order_items.withColumn("LoadingTime", current_timestamp())




df_order.writeStream \
            .format("Delta") \
            .option("checkpointLocation","/Volumes/ecommerce/data/streaming_data/order_checkpoint/") \
            .option("mode","append") \
            .trigger(once = True) \
            .toTable("ecommerce.bronze.brz_orders") \
            .awaitTermination()


df_order_items.writeStream \
            .format("delta") \
            .option("checkpointLocation","/Volumes/ecommerce/data/streaming_data/Order_items_checkpoints/") \
            .option("mode","append") \
            .trigger(once = True) \
            .toTable("ecommerce.bronze.brz_order_items") \
            .awaitTermination()



In [0]:
%sql
select * from ecommerce.bronze.brz_orders

In [0]:
%sql
select * from ecommerce.bronze.brz_order_items